In [ ]:
#@title ◢ Git clone and install DeOldify
%cd /content/
#!git clone https://github.com/jantic/DeOldify.git DeOldify 
!git clone https://github.com/styler00dollar/Colab-DeOldify DeOldify
%cd DeOldify
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')
!pip install -r colab_requirements.txt
import fastai
from deoldify.visualize import *
!mkdir 'models'

#!wget https://deepai-opensource-codebases-models.s3-us-west-2.amazonaws.com/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth
!wget https://www.dropbox.com/s/usf7uifrctqw9rl/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen.pth
!wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png

colorizer = get_image_colorizer(artistic=True)

In [ ]:
%cd /content/DeOldify
source_url = 'https://i.pinimg.com/originals/f1/dd/4c/f1dd4ca98416ae0db5061587810e7fd5.jpg' #@param {type:"string"}
render_factor = 35  #@param {type: "slider", min: 7, max: 40}
watermarked = True #@param {type:"boolean"}

if source_url is not None and source_url !='':
    image_path = colorizer.plot_transformed_image_from_url(url=source_url, render_factor=render_factor, compare=True, watermarked=watermarked)
    show_image_in_notebook(image_path)
else:
    print('Provide an image url and try again.')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os

Mounted at /content/drive


In [4]:
#build simple pandas frame with details about what file were processed.
import pandas as pd
import numpy as np

dtypes = np.dtype([
    ('date', np.datetime64),
    ('img_name', str),
    ('size', float),
    ('dimension', str)
])

image_df = pd.DataFrame(np.empty(0, dtype=dtypes))

print(image_df)

Empty DataFrame
Columns: [date, img_name, size, dimension]
Index: []


In [5]:
from datetime import datetime

current_datetime = datetime.now()
test_name_dict = {'E1':{'date':current_datetime, 'img_name':'test1.jpg', 'size':7.3,'dimension':'480x310'},
                  'E2':{'date':current_datetime, 'img_name':'test2.jpg', 'size':7.9,'dimension':'480x250'},
                  'E3':{'date':current_datetime, 'img_name':'test3.jpg', 'size':7.5,'dimension':'480x450'} }

image_df = image_df.append({'date': current_datetime, 'img_name': 'test_img', 'size':11.2, 'dimension':'860x480'}, ignore_index=True)

for i_key, val in test_name_dict.items():
  image_df = image_df.append({'date': val['date'], 'img_name': val['img_name'], 'size':val['size'], 'dimension':val['dimension']}, ignore_index=True)

print(image_df)

                        date   img_name  size dimension
0 2022-12-29 05:27:43.639281   test_img  11.2   860x480
1 2022-12-29 05:27:43.639281  test1.jpg   7.3   480x310
2 2022-12-29 05:27:43.639281  test2.jpg   7.9   480x250
3 2022-12-29 05:27:43.639281  test3.jpg   7.5   480x450


In [8]:
import os

import enum

# Enum for size units
class SIZE_UNIT(enum.Enum):
   BYTES = 1
   KB = 2
   MB = 3
   GB = 4


def convert_unit(size_in_bytes, unit):
   """ Convert the size from bytes to other units like KB, MB or GB"""
   if unit == SIZE_UNIT.KB:
       return size_in_bytes/1024
   elif unit == SIZE_UNIT.MB:
       return size_in_bytes/(1024*1024)
   elif unit == SIZE_UNIT.GB:
       return size_in_bytes/(1024*1024*1024)
   else:
       return size_in_bytes


def get_file_size(file_name, size_type = SIZE_UNIT.BYTES ):
   """ Get file in size in given unit like KB, MB or GB"""
   size = os.path.getsize(file_name)
   return convert_unit(size, size_type)

   

In [ ]:
#Make sure that we have the correct working directory set
%cd /content/drive/MyDrive/Github/bw_photos/

#get a list of all the files and directories within the correct working directory
files_dir = os.listdir()

base_folder_path = os.getcwd()
print(f'Working in this current directory: {base_folder_path}')
for single_file_obj in files_dir:
  if( single_file_obj.endswith('.jpg') ):
    #Now get just the files that are jpg for processing.
    print(f'found a jpg file for processing: {single_file_obj}')
    
    #taking jpg file that have spaces between words and replacing with underscore
    processed_file_nm = single_file_obj.replace(' ', '_')
    color_output_full_file_nm = f'color_{processed_file_nm}'
    color_full_path = os.path.join(base_folder_path,'colorized',color_output_full_file_nm)
    print(f'Here is where the processed file is being put: {color_full_path}')

    #Now do the actually color processing.
    bw_full_file_path = os.path.join(base_folder_path,single_file_obj)
    print(f'Here is the bw path: {bw_full_file_path}')
    img_pic = colorizer.plot_transformed_image(bw_full_file_path, render_factor=35, display_render_factor=True, figsize=(10,10))
    #print(type(img_pic))

    #Now build the full file path where it is in the results and copy to drive.
    full_results_loc = os.path.join(base_folder_path,'result_images', single_file_obj)
    #Now copy
    !cp "$full_results_loc" $color_full_path

    print('Finished moving to the next one **********************************************************************************')


In [12]:
from PIL import Image

img_filename = '/content/drive/MyDrive/Github/bw_photos/colorized/color_img321.jpg'
with Image.open(img_filename) as image:
    width, height = image.size
    print(f'Here is the width: {width}. Here is the height: {height}')


# get file size in KB
size = get_file_size(img_filename, SIZE_UNIT.KB)
print('Size of file is : ', size ,  'KB')

rnd_size = round(size,3)
print(f'Here is the value that is rounded: {rnd_size} KB')

Here is the width: 2856. Here is the height: 2256
Size of file is :  974.7978515625 KB
Here is the value that is rounded: 974.798 KB


In [15]:

from datetime import datetime


col_dtypes = np.dtype([
    ('date', np.datetime64),
    ('img_name', str),
    ('size', float),
    ('unit_size', str),
    ('dimension', str)
])


#Make sure that we have the correct working directory set
#Going to do book-keeping on the color phots.
%cd /content/drive/MyDrive/Github/bw_photos/colorized/

image_df = pd.DataFrame(np.empty(0, dtype=col_dtypes))
current_datetime = datetime.now()

#get a list of all the files and directories within the correct working directory
color_files_dir = os.listdir()

base_folder_path = os.getcwd()
print(f'Working in this current directory: {base_folder_path}')
for single_file_obj in color_files_dir:
  if( single_file_obj.endswith('.jpg') ):
    #Now get just the files that are jpg for processing.
    print(f'found a color jpg file for processing: {single_file_obj}')
    
    #Get the size and dimension for the file for bookkeeping
    color_img_fl = os.path.join(base_folder_path, single_file_obj)

    #Get the size of the image.
    color_img_size = None
    with Image.open(color_img_fl) as image:
      color_img_dim = image.size

    formatted_color_img_size = round( get_file_size( color_img_fl, SIZE_UNIT.KB), 3)


    #Put requested info into dataframe
    image_df = image_df.append({'date': current_datetime, 'img_name': single_file_obj, 'size':formatted_color_img_size, 'unit_size': 'KB', 'dimension':color_img_dim}, ignore_index=True)

    

    print('Finished moving to the next one **********************************************************************************')

/content/drive/MyDrive/Github/bw_photos/colorized
Working in this current directory: /content/drive/MyDrive/Github/bw_photos/colorized
found a color jpg file for processing: color_img073.jpg
Finished moving to the next one **********************************************************************************
found a color jpg file for processing: color_Klempel_Family_Homestead.jpg
Finished moving to the next one **********************************************************************************
found a color jpg file for processing: color_Herman_Nollmeyer.jpg
Finished moving to the next one **********************************************************************************
found a color jpg file for processing: color_Girls_at_homestead.jpg
Finished moving to the next one **********************************************************************************
found a color jpg file for processing: color_Katze.jpg
Finished moving to the next one ******************************************************

In [16]:
print(image_df)

                        date                            img_name     size  \
0 2022-12-29 06:19:46.398648                    color_img073.jpg  443.779   
1 2022-12-29 06:19:46.398648  color_Klempel_Family_Homestead.jpg  202.914   
2 2022-12-29 06:19:46.398648          color_Herman_Nollmeyer.jpg  229.178   
3 2022-12-29 06:19:46.398648        color_Girls_at_homestead.jpg  159.521   
4 2022-12-29 06:19:46.398648                     color_Katze.jpg  328.864   
5 2022-12-29 06:19:46.398648                color_Katze_Kids.jpg  207.664   
6 2022-12-29 06:19:46.398648                    color_img317.jpg   79.865   
7 2022-12-29 06:19:46.398648                    color_img321.jpg  974.798   

  unit_size     dimension  
0        KB  (1752, 2370)  
1        KB   (1314, 767)  
2        KB   (972, 1572)  
3        KB   (1140, 846)  
4        KB  (1110, 1428)  
5        KB   (978, 1566)  
6        KB    (552, 852)  
7        KB  (2856, 2256)  
